# Session de Recrutement Printemps 2017 - Team Tax Analytics
## Rendu de Guillaume Grelet - guillaume.grelet@telecom-paristech.fr

Date : février 2017


Ce rendu a pour but de fournir une réponse à l'exercice posé par PwC sur la gestion de tableur Excel.  
Il utilise les packages suivante : jupyter, pandas, PyQt, ipywidgets  
  
  
Notes :
Le code suivant est systématiquement commenté en anglais, comme l'usage le veut.  


In [159]:
# Import the necessary packages..

# For the widgets
%gui qt 
from IPython.display import display, Markdown, Latex
from PyQt5.QtWidgets import QFileDialog
from ipywidgets import *

# For the excel file processing...
import pandas as pd
import numpy as np

# For the Jaro-Winkler distance calculation
import jellyfish

def filesearch(dir=None):
    """
    That function allows the user to select a file via a PyQt5 dialog window and return the file name as a str
    """
    if dir is None: dir ="~/" # Default path is ~/
    filename = QFileDialog.getOpenFileName(None, "Select data file...", 
            dir, filter="SM Files (*.xlsx)")
    return filename[0]

In [169]:
datapath = filesearch("~/")
display(Markdown("## Cette fonction constitue la réponse à la question 1 de l'exercice"))

## Cette fonction constitue la réponse à la question 1 de l'exercice

In [165]:
#%%time
print("Processing file...")
data = pd.read_excel(datapath) # Data reading
print("\nProcessing OK.\n")
display(data, Markdown("## Ci-dessus les données brutes"))

Processing file...

Processing OK.



,Pays,Marchandises,Industries,Nombre d'employés,CA,Exportations,pib
0,France,Hélicopère,Aero,22000,3 368 000 000,1000,2421
1,It,Ferrari,Automobile,2800,290000000,600,1815
2,Allemagne,A320,Aeronautique,20000,6789000000,2000,3357
3,Bulgarie,Electricité,Energy,35000,23000000,12,48
4,UK,engines,Auto,20000,7984000000,1400,2849
5,Sverige,hydrolique,energie,10000,400000000,300,482
6,Finland,nokia,Telecommunication,5000,30000000,100,229
7,Norvège,pétrole,energ,3000,900000000,90,389
8,Danemark,Gaz naturel,energie,4000,24000000,95,294
9,Spain,mécanique de précison,Auto,10000,23400000,500,1192


## Ci-dessus les données brutes

In [166]:
zero = []

# Find NaN lines
for k in range(0, len(data.index)):
    zero = zero + [all(data.ix[k].isnull())]
zero = pd.Series(zero)

# Make partition of faile depending on the NaN lines
partitions = (zero != zero.shift()).cumsum()
split = data[np.invert(zero)].groupby(partitions)
keys = split.groups.keys()

# Find the biggest piece of file
x = []
for key in keys:
    x = x + [len(split.get_group(key).index)]
x
data2 = data
data = split.get_group(np.argmax(x) + 1)
data
display(data, Markdown("## Le jeu de donnée ci dessus correspond au plus grand rectangle de données cohérentes, cela répond à la question 2"))

,Pays,Marchandises,Industries,Nombre d'employés,CA,Exportations,pib
0,France,Hélicopère,Aero,22000,3 368 000 000,1000,2421
1,It,Ferrari,Automobile,2800,290000000,600,1815
2,Allemagne,A320,Aeronautique,20000,6789000000,2000,3357
3,Bulgarie,Electricité,Energy,35000,23000000,12,48
4,UK,engines,Auto,20000,7984000000,1400,2849
5,Sverige,hydrolique,energie,10000,400000000,300,482
6,Finland,nokia,Telecommunication,5000,30000000,100,229
7,Norvège,pétrole,energ,3000,900000000,90,389
8,Danemark,Gaz naturel,energie,4000,24000000,95,294
9,Spain,mécanique de précison,Auto,10000,23400000,500,1192


## Le jeu de donnée ci dessus correspond au plus grand rectangle de données cohérentes, cela répond à la question 2

In [170]:
# Texterea widget
input_label_widget = widgets.Textarea(
    placeholder='Place here the preprocessed labels',
    value='Pays\nMarchandises',
    disabled=False
)

# Button widget
button1 = widgets.Button(
    description='Save',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)


display(Markdown("## Remplissez ci-dessus la liste des labels standards préprogrammés. Cette fonction répond à la question 3"), input_label_widget, button1)
input_label_list = []
def handle_submit(sender):
    global x
    x = input_label_widget.value
    global input_label_list
    input_label_list = x.split()
    print("input_label_list = ")
    print(input_label_list)
    print("Fichier sauvé")
    return None

def when_button_clicked(button1):
    out = handle_submit(input_label_widget)
    return None 

button1.on_click(when_button_clicked)

## Remplissez ci-dessus la liste des labels standards préprogrammés. Cette fonction répond à la question 3

In [131]:
found_labels = list(data)
for str in found_labels:
    print("    " + str)
display(Markdown("## Nous avons trouvé les labels ci-dessus"))

    Pays
    Marchandises
    Industries 
    Nombre d'employés 
    CA
    Exportations
    pib


## Nous avons trouvé les labels ci-dessus

In [133]:
intersection = list(set(found_labels).intersection(input_label_list))
#print(intersection)

existing_label_widget = widgets.Textarea(
    value="\n".join(intersection) + "\n",
    placeholder='Type something',
    disabled=True
)

display(Markdown("## Les labels ci-dessus sont issus du fichier Excel et existent dans la liste préprogrammée"), existing_label_widget)

## Les labels ci-dessus sont issus du fichier Excel et existent dans la liste préprogrammée

In [147]:
new_labels = []
for str in found_labels:
    if str not in intersection:
        new_labels.append(str)
                
preprocessed_label_widget = widgets.Textarea(
    value="\n".join(new_labels) + "\n",
    placeholder='Type something',
    disabled=False
)

button2 = widgets.Button(
    description='Ok',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)

display(Markdown("## Les labels ci-dessus ne sont pas dans la liste standardisée et sont donc nouveaux. Vous pouvez modifier la liste et les ajouter"), preprocessed_label_widget, button2)

def handle_submit(sender):
    global y
    y = preprocessed_label_widget.value
    global modified_list
    modified_list = y.splitlines()
    print("modified_list = ")
    print(modified_list)
    return None

def when_button_clicked(button):
    out = handle_submit(preprocessed_label_widget)
    return None 

button2.on_click(when_button_clicked)

## Les labels ci-dessus ne sont pas dans la liste standardisée et sont donc nouveaux. Vous pouvez modifier la liste et les ajouter

modified_list = 
['Industries ', "Nombre d'employés ", 'CA', 'Exportations', 'pib']


In [154]:
mat_size = (len(modified_list) + len(input_label_list)), (len(found_labels))
mat0 = np.array([modified_list + input_label_list])

mat1 = np.array([[None] +  modified_list + input_label_list])
mat2 = np.array([found_labels]).T
mat3 = np.zeros(mat_size)
a, b = 0, 0
for str1 in modified_list + input_label_list:
    for str2 in found_labels:
        mat3[a, b] = round(jellyfish.jaro_winkler(str1, str2), 3)
        a = a + 1
    a = 0
    b = b + 1
mat4 = np.concatenate((mat2, mat3), axis = 1)
mat5 = np.concatenate((mat1, mat4), axis = 0)

In [150]:
print(mat5)

[[None 'Industries ' "Nombre d'employés " 'CA' 'Exportations' 'pib' 'Pays'
  'Marchandises']
 ['Pays' '0.447' '0.0' '0.0' '0.444' '0.0' '1.0' '0.444']
 ['Marchandises' '0.645' '0.435' '0.0' '0.478' '0.0' '0.444' '1.0']
 ['Industries ' '1.0' '0.444' '0.0' '0.482' '0.0' '0.447' '0.645']
 ["Nombre d'employés " '0.444' '1.0' '0.0' '0.519' '0.463' '0.0' '0.435']
 ['CA' '0.0' '0.0' '1.0' '0.0' '0.0' '0.0' '0.0']
 ['Exportations' '0.482' '0.519' '0.0' '1.0' '0.472' '0.444' '0.478']
 ['pib' '0.0' '0.463' '0.0' '0.472' '1.0' '0.0' '0.0']]


In [294]:
treshold = 0.8
k = 0
a = np.where(mat3>treshold)
matched_labels = found_labels
print("Recherche de labels similaires")
while k < np.maximum(len(a[0]),len(a[1])):
    print("    ------>  " + mat2[a[0][k], 0] + " ressemble à " + mat0[0,a[1][k]])
    for x in matched_labels:
        if x == mat2[a[0][k], 0]:
            matched_labels[matched_labels.index(mat2[a[0][k], 0])] = mat0[0,a[1][k]]
    k = k + 1
print("Labels similaires remplacés")



Recherche de labels similaires
    ------>  axe ressemble à axer
    ------>  sous_axe ressemble à sous_axe
    ------>  code_hospidiag ressemble à codes_hospidiag
Labels similaires remplacés


In [151]:
dico = {}

linker_label_widget = widgets.Text(
    value='Autos',
    placeholder='Type something',
    disabled=False
)

linked_label_widget = widgets.Textarea(
    value="Automobiles\nVoiture\nCar",
    placeholder='Type something',
    disabled=False
)

button3 = widgets.Button(
    description='Save',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)

display(Markdown("## Vous pouvez modifier la liste et les ajouter pour lier"), Markdown("## Vous pouvez le faire autant de fois que vous voulez"), linker_label_widget, linked_label_widget, button2)

def handle_submit(sender):
    global linker
    linker = linker_label_widget.value
    global links
    links = linked_label_widget.value.split()
    global dico
    di = {linker: links}
    dico.update(di)
    print(dico)
    return None

def when_button_clicked(button):
    out = handle_submit(linked_label_widget)
    return None 



## Vous pouvez modifier la liste et les ajouter pour lier

## Vous pouvez le faire autant de fois que vous voulez